## Fonctions normalisation, génération des fichiers, dataset

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import MinMaxScaler

In [ ]:
years_to_import = [1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 
                   1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2021] # JO de 2020 ont eu lieu en 2021

In [ ]:
years_to_export = [1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 
                   1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

In [ ]:
def convert_to_numeric(value):
    """
    Converts strings with 'B', 'M', or 'k' suffixes to numeric values.
    
    Parameters:
    value (str or float): The value to convert.

    Returns:
    float: The numeric value.
    """
    if isinstance(value, str):
        if 'B' in value:
            return float(value.replace('B', '')) * 1_000_000_000
        elif 'M' in value:
            return float(value.replace('M', '')) * 1_000_000
        elif 'k' in value:
            return float(value.replace('k', '')) * 1_000
    return float(value)  # If it's already numeric or doesn't have suffixes

In [ ]:
def normalize_data(data, exclude_columns=[]):
    """
    Normalizes the numeric columns of a dataset to a range between 0 and 1 using Min-Max Scaling.
    
    Parameters:
    data (pd.DataFrame): The dataset to normalize.
    exclude_columns (list): List of column names to exclude from normalization (e.g., non-numeric columns).

    Returns:
    pd.DataFrame: The dataset with normalized numeric columns.
    """
    # Convert columns to numeric if they contain "B", "M", or "k"
    for col in data.columns:
        if col not in exclude_columns:
            data[col] = data[col].apply(convert_to_numeric)
    
    # Separate excluded columns (e.g., country names) and numeric columns
    data_to_normalize = data.drop(columns=exclude_columns)
    data_excluded = data[exclude_columns]
    
    # Apply Min-Max Scaling to numeric columns
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data_to_normalize)
    
    # Convert the normalized data back to a DataFrame
    normalized_df = pd.DataFrame(normalized_data, columns=data_to_normalize.columns, index=data.index)
    
    # Combine excluded columns with the normalized data
    final_data = pd.concat([data_excluded, normalized_df], axis=1)
    
    return final_data

In [ ]:
# import pdb # pdb pour debug interactif (utile ?)

"""
Fonction pour générer les fichiers en utilisant un tableau d'années pour nommer les fichiers.
"""
def generate_files_by_year(data, path, suffix, years_to_export):
    # Vérifie que la longueur de years correspond au nombre de colonnes de data (moins la colonne 'country')
    if len(years_to_export) != len(data.columns) - 1:
        raise ValueError("Projet GML : La longueur du tableau 'years' doit être égale au nombre de colonnes d'années dans 'data'.")
    
    os.makedirs(path, exist_ok=True) # Crée le répertoire si nécessaire
    
    # Itère sur les colonnes du DataFrame et les années fournies simultanément
    for col, year in zip(data.columns[1:], years_to_export):  # data.columns[1:] ignorer la 1ère colonne ('country')
        # Assurer que year est une chaîne de caractères pour la génération de fichier
        year_str = str(year) # Convertit l'année en chaîne de caractères

        year_data = data[['country', col]]  # Sélectionne la colonne 'country' et la colonne de l'année actuelle
        filename = os.path.join(path, f"{year_str}{suffix}.csv")
        year_data.to_csv(filename, index=False)  # Sauvegarde du fichier csv

In [ ]:
"""
Fonction pour générer un array de dataframes à partir des fichiers
"""
def generate_df_from_files(path, suffix):

    if not os.path.exists(path):
        raise ValueError(f"The directory {path} does not exist.")
    
    file_list = glob.glob(os.path.join(path, f"*{suffix}.csv"))
    dataframes = []

    for file in file_list:
        df = pd.read_csv(file)
        dataframes.append(df)

    return dataframes

In [ ]:
"""
Function to load an array of rows (entire rows as individual elements) from a single CSV file.
"""
def generate_df_labels_from_file(file_path):
    if not os.path.exists(file_path):
        raise ValueError(f"The file {file_path} does not exist.")
    
    # Load the CSV into a DataFrame
    df = pd.read_csv(file_path)
    
    # Initialize an array to store rows
    rows_array = []

    # Append each entire row as a separate element in the list
    for _, row in df.iterrows():
        rows_array.append(row)

    return rows_array